# Hito 3: Corona virus (Covid-19)

## Presentación

Somos el equipo conformado por Raúl Cid, José Espina, Michelle Valenzuela y Alejandro Veragua. Nuestro *dataset* corresponde al [publicado en el portal Kaggle](https://www.kaggle.com/sudalairajkumar/novel-corona-virus-2019-dataset) construido y mantenido por la Universidad John Hopkins, al igual que para el hito 1

## Comentarios sobre el hito 1

Se realizó un análisis exploratorio, donde descubrimos varios datos inconsistentes y outliers
Se buscaron correlaciones contra atributos como el ingreso per cápita, la temperatura, el índice de pobreza de los diferentes países, pero sin éxito. Finalmente se realizó una prueba de concepto de clústering a partir de los coeficientes de un modelo de regeresión polinomial donde sí aparecieron resultados interesantes

## Propuesta del hito 3
Considerando que los polinomios hacen "overfitting" cuando el grado es muy alto, como el ejercicio realizado en el hito 1, aprovecharemos hacer algo similar, con la diferencia de usar un modelo auto-regresivo y hacer clustering sobre los coeficientes que resulten del modelo. En particular, quisimos experimentar con un modelo auto-regresivo con media móvil (ARMA). Como se tienen datos sólo de los últimos meses y el virus es nuevo, los datos no son estacionarios (o no hay suficientes datos para demostrar que lo son), por lo que usaremos una variante llamada ARIMA (donde la "i" hace referencia a la integración), que permite encontrar las diferencias no-estacionarias para lograr la estacionariedad (https://www.statisticshowto.com/arma-model/)

## Metodología de trabajo para el hito 3
* se preprocesaron los datos, removiendo los datos outliers, ya que descubrimos en el hito 1 que hay paises con datos demasiado alejados de la tendencia central

Además, se refuerza la idea de 2 artículos publicados que usan el mismo acercamiento
1) Benvenuto, D., Giovanetti, M., Vassallo, L., Angeletti, S., & Ciccozzi, M. (2020). Application of the ARIMA model on the COVID-2019 epidemic dataset. Data in brief, 105340.
2) Yang, Q., Wang, J., Ma, H., & Wang, X. (2020). Research on COVID-19 Based on ARIMA ModelΔ—Taking Hubei, China as an example to see the epidemic in Italy. Journal of Infection and Public Health.

NOTA IMPORTANTE: Para ejecutar este notebook, se necesitan scikit, matplotlib, numpy y statsmodels. Éste último para poder aplicar el modelo ARIMA. Se debe instalar con pip en el ambiente del libro:

$ pip install statsmodels

o con conda:

$ conda install -c conda-forge statsmodels

(https://www.statsmodels.org/stable/install.html)

NOTA IMPORTANTE 2: Calcular el modelo ARIMA toma un tiempo de cómputo importante

En detalle, el plan es:

* Calcular ARIMA para todos los países


## Configuración base del notebook
El bloque a continuación configura el libro, definiendo funciones y cargando las fuentes de datos

In [145]:
# Librería de álgebra lineal
import numpy as np 
# Procesamiento de datos y carga de archivos CSV
import pandas as pd
# Librearía gráfica
import matplotlib.pylab as plt
# Librería para usar recursos del sistema operativo
import os 
# Librearía usada para extraer el nombre del archivo de cada path
import ntpath
# Librería gráfica
import seaborn as sns
# Librearía para usar herramientas relacionadas con
# aprendizaje máquina. Se usó para preprocesar y
# calcular regresión
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
# Colores
from matplotlib import colors
# Ticker
import matplotlib.ticker as ticker
# Animation
import matplotlib.animation as animation
# Arima
from statsmodels.tsa.arima_model import ARIMA
from matplotlib import pyplot
from sklearn.metrics import mean_squared_error
# Configuración general
%load_ext autoreload
%autoreload 2

# Matplotlib inline
%matplotlib notebook
%matplotlib inline

# Se prepara carga de los archivos del dataset
paths = []
for dirname, _, filenames in os.walk('./novel-corona-virus-2019-dataset'):
    for filename in filenames:
        path = os.path.join(dirname, filename)
        paths.append(path);
dataFrames = {}
for path in paths:
    dataFrames[ntpath.basename(path)] = pd.read_csv(path)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Preproceso: Outliers
Como parte del preprocesamiento, se realiza el siguiente tratamiento a los datos: Para cada país, se revisan las columnas de confirmados, fallecidos y recuperados. En cada una de ellas se buscan outliers, en este caso, valores que superen en magnitud al percentil 99 o que sean menores a cero. Estos se transorman a NaN y luego interpolamos los datos para llenar estos espacios.
Además, se debe recalcular la cantidad acumulada por día para cada columna.

In [153]:
dset_base = pd.read_csv('dataset.csv')
for pais in dset_base['pais'].unique():
    for col in ['num_confirmados', 'num_fallecidos', 'num_recuperados']:
        pais_col = dset_base[dset_base['pais']==str(pais)][col]
        changed_values = pd.Series(np.nan, index=pais_col.index)
        for value in pais_col.iteritems():
            if value[1] > pais_col.quantile(.99) or value[1] < 0:
                changed_values[value[0]] = np.nan
            else:
                changed_values[value[0]] = value[1]
        changed_values.interpolate(inplace=True)
        acc = changed_values.cumsum()
        dset_base.loc[(dset_base['pais']==pais), col] = changed_values
        dset_base.loc[(dset_base['pais']==pais), str('acc') + col[3:]] = acc

## Modelo ARIMA
Usamos un modelo ARIMA p=5,d=1,q=0. El detalle de estos parámetros se explica a continuación (basado en https://machinelearningmastery.com/arima-for-time-series-forecasting-with-python/)

* El parámetro "p" es el número de observaciones pasadas que considera el modelo. También se le llama "orden". Usamos  14 debido a que es el tiempo que demora una persona es saber que tiene el virus, considerando tiempos de síntomas y resultado del exámen. Además, 14 días es el tiempo que dura la cuarentena para las personas con resultado positivo en su exámen
* El parámetro "d" es el grado de diferenciación. Esto se usa para generar estacionariedad en series no estacionarias
* El parámetro "q" es el tamaño de la media móvil para el error de los datos anteriores

In [ ]:
# Se calculan coeficientes del ARIMA por país. Si ya está calculado se lee el csv
dframe_arima = None
if os.path.isfile('arima_coefs.csv') :
    dframe_arima = pd.read_csv('arima_coefs.csv')
else :
    pais_arima_coef = dict()
    paises_error = []
    for pais in dset_base.pais.unique() :
        dbset_actual = dset_base[dset_base['pais']==pais][['num_dia_desde_primer_caso','num_confirmados']]
        dbset_actual= dbset_actual.reset_index()
        series_actual = pd.Series(dbset_actual.num_confirmados.values, index=dbset_actual.num_dia_desde_primer_caso)
        series_actual.index = pd.DatetimeIndex(series_actual.index).to_period('D')
        try :
            model = ARIMA(series_actual.astype(float), order=(5,1,0),missing="drop")
            model_fit = model.fit(disp=False)
            pais_arima_coef[pais] = model_fit.params
        except :
            paises_error.append(pais)
    dframe_arima = pd.DataFrame(data=pais_arima_coef)
    dframe_arima.to_csv('arima_coefs.csv')
print(dframe_arima)

Con el fin de hacer una revision a los resultados del clustering se procede a realizar un analisis de las tendencias de las  caracteristicas de cada uno de los clusters. 

In [ ]:
clustering = None
values= np.unique(clustering)
#ex: clustering= agglomerative_clusters 
values= np.unique(agglomerative_clusters)

data_agl = features_df.copy()
data_agl['cluster_num'] = agglomerative_clusters #### 
data_agl = data_agl[['region', 'pais', 'cluster_agg']]
data_agl_comp = pd.merge(data_agl, dset_extra, how='inner')
data_agl_comp = dset_base.merge(data_agl_comp.set_index('pais'), on='pais') #variable sera renombrada
# de esto lo que importa es la ultima linea (que es la data a trabajar), data_clusters contiene todos los atributos:
data_clusters = 'la data que deberia tener uwu'

datas={}
for i in range(0, len(values)): #separamos y guardamos en diccionario
    llave=values[i]
    datai = data_clusters.loc[data_clusters['cluster_num']==values[i]]
    datas[llave]=datai

interes=[ 'gdp_rank', 'gdp_usd', 'gdp_en_salud', 'edad_media', 'gini', 'poblacion', 'taza_muerte1000' ]
colores= ['darkorange', 'springgreen', 'lightseagreen', 'orchid', 'chocolate', 'lightsteelblue', 'indianred', 'cadetblue', 'gold']

#estadisticos generales
for i in range(0,len(values)):
    print('info gnral cluster ', values[i], ':')
    for a in range(0,len(interes)):
        datos=datas[values[i]]
        print(interes[a], 'promedio: ', datos[interes[a]].mean(), ';  desviacion:   ', datos[interes[a]].std() )
        print('---------------------------------------')
    print('*******************', 'fin cluster <', values[i] ,'>************************')


#paises en el cluster
for i in range(0,len(values)):
    print('cluster', values[i])
    data=datas[values[i]]
    column_valxs = data[["pais"]].values
    unique_values =  np.unique(column_valxs)    
    print(unique_values)
    print('-----------------------------------------------------------------------------')

#boxplots
for col in interes:
    par=col
    data_clusters.groupby('cluster_num')
    f, ax = plt.subplots(figsize=(7,5))
    img=sns.boxplot(y=par, x= 'cluster_num', data=data_clusters, width=0.3, showfliers = True) ##showf en false no muestra outL

#densidades (permite apreciar mejor las tendencias)
for col in interes:
    par=col
    plt.figure()
    plt.title(col)
    for i in range(0,len(datas)):
        datos=datas[i]
        datos=list(datos[col])
        sns.distplot(datos, hist=False, color=colores[i])
    plt.close

#Matrices de correlacion
for i in range(0,len(values)):
    print('para cluster:  ', values[i])
    data=datas[values[i]]
    mm_scaler = preprocessing.MinMaxScaler()
    data = data.drop(["fecha_observacion"  ,"region_x", "region_y","cluster_num", "pais", "num_dia_desde_22_enero", "num_dia_desde_primer_caso" ],axis=1)
    #quiza eso se pueda cambiar, lo de arriba
    scaled_at = mm_scaler.fit_transform(data)
    scaled = pd.DataFrame(scaled_at)
    cormatrix = scaled.corr()
    f, ax = plt.subplots()
    sns.heatmap(cormatrix,vmax=1,vmin=-1,center=0, cmap='rainbow' )

#Algunas curvas en el cluster:
for i in values:
    print('llave: ', i)
    dato=datas[i]
    column_valxs = dato[["pais"]].values
    muestra = random.choices(np.unique(column_valxs), k=7)
    k=0
    var='num_dia_desde_primer_caso'
    plt.figure()
    plt.title('cluster'+ str(i))
    for k in range(0,len(muestra)):
        pais=muestra[k]
        print('pais: ', pais)
        datos=data_clusters.loc[data_clusters['pais']==pais]
        sns.lineplot(x=var, y='acc_confirmados', data=datos, color=colores[k])
    plt.close  

**Discusion y conclusiones**

-Respecto al clustering:

Las feautures entregadas por el modelo AR parecieran no ser de mucha utilidad para realizar clustering. Como se observa en el analisis de caracteristicas de cada cluster, no existen evidencias o indicios suficientes para asegurar la coherencia del agrupamiento. No se observan patrones o tendencias claras respecto del tipo de curva que se describe, asimismo tampoco existen resultados significativos en cuanto a los indicadores caracteristicos de paises.

-Respecto al modelo:

-Respecto a los datos:

Como se puede observar en este informe, fue necesario realizar una gran cantidad de pre-procesamiento y limpieza a los datos debido a que existian una gran cantidad de inconsistencias (ruido) generado por la forma en que los paises han estado recopilando la informacion. Por ejemplo se tiene el caso de Chile donde se podia apreciar un peak en la columna de casos confirmados (saltaba de 5000 a 35000 aprox), este pico se debia a 'casos pendientes' que el minsal nohabia reportado. Para varios paises y otras columnas existian problemas de indole similar. 
Fue de gran relevancia la limpieza de los datos debido a que podian generarse clusters que agrupasen estos ruidos.

**Direcciones futuras:**

